# Combining VBF and ggF templates

Need to update this since pass regions' names have changed to pass_ggf and pass_vbf even for separate templates

Author(s): Raghav Kansal

In [ ]:
import pickle
import json
from pathlib import Path
from HHbbVV.hh_vars import years

In [ ]:
ggf_templates_path = Path("templates/24Apr9ggFMP9965")
vbf_templates_path = Path("templates/24Apr8VBFHP999")
templates_path = Path("templates/24Apr10NonresCombined")
templates_path.mkdir(exist_ok=True)

## Combining templates

In [ ]:
for year in years:
    combined_templates = {}

    with (ggf_templates_path / f"{year}_templates.pkl").open("rb") as f:
        ggf_templates = pickle.load(f)

    with (vbf_templates_path / f"{year}_templates.pkl").open("rb") as f:
        vbf_templates = pickle.load(f)

    for region, h in ggf_templates.items():
        if region.startswith("pass"):
            # rename pass regions
            combined_templates[region.replace("pass", "pass_ggf")] = h
        else:
            combined_templates[region] = h

    for region, h in vbf_templates.items():
        if region.startswith("pass"):
            # rename pass regions
            combined_templates[region.replace("pass", "pass_vbf")] = h
        else:
            # checking that fail regions are identical
            assert combined_templates[region] == h

    # save combined templates
    with (templates_path / f"{year}_templates.pkl").open("wb") as f:
        pickle.dump(combined_templates, f)

## Combining systematics

In [ ]:
combined_systematics = {"pass_ggf": {}, "pass_vbf": {}}

with (ggf_templates_path / "systematics.json").open("r") as f:
    ggf_systematics = json.load(f)

with (vbf_templates_path / "systematics.json").open("r") as f:
    vbf_systematics = json.load(f)

for skey, sval in ggf_systematics.items():
    if skey in years:
        combined_systematics[skey] = {}
        for region, val in sval.items():
            if region.startswith("pass"):
                # rename pass regions
                combined_systematics[skey][region.replace("pass", "pass_ggf")] = val
            else:
                combined_systematics[skey][region] = val
    else:
        # LP SFs
        combined_systematics["pass_ggf"][skey] = sval

for skey, sval in vbf_systematics.items():
    if skey in years:
        for region, val in sval.items():
            if region.startswith("pass"):
                # rename pass regions
                combined_systematics[skey][region.replace("pass", "pass_vbf")] = val
            else:
                # checking that fail regions are identical
                assert combined_systematics[skey][region] == val
    else:
        # LP SFs
        combined_systematics["pass_vbf"][skey] = sval

with open(templates_path / "systematics.json", "w") as f:
    json.dump(combined_systematics, f, indent=4)

Combining cutflows